In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from datetime import datetime
from IPython.display import display

## Read Excel files

In [ ]:
# Assuming files are in the same directory
# Order is not important, as they will be sorted further down
input_names = ["ArcGISMonitor_2022_04_17_05_02_07.xlsx", 
                "ArcGISMonitor_2022_04_24_05_02_23.xlsx", 
                "ArcGISMonitor_2022_05_01_05_02_39.xlsx", 
                "ArcGISMonitor_2022_05_08.xlsx", 
                "ArcGISMonitorMay9-12.xlsx"]
input_xl = []
for n in input_names:
    input_xl.append(pd.ExcelFile(n))

## Read all sheets in all Excel files

In [ ]:
input_logs = []
for idx, n in enumerate(input_names):
    input_log = {}
    for s in input_xl[idx].sheet_names:
        if s in ['Summary', 'Inputs']:
            df = pd.read_excel(n, sheet_name=s, skiprows=1)
        else:
            df = pd.read_excel(n, sheet_name=s, skiprows=2, index_col=0)
        input_log[s] = df
    v = input_log["Inputs"].at[6, "Value"]
    input_log["Inputs"].loc[input_log["Inputs"].index[6], "Value"] = datetime.strptime(v, "%Y-%m-%d %H:%M:%S%z")
    input_logs.append(input_log)

# Sort the loaded data by date, so the earliest info is first
def sortByEndDate(e):
    return e["Inputs"].at[6, "Value"]

input_logs.sort(key=sortByEndDate)

### Constants

In [ ]:
# Set up constants
CRITICAL = '■'
WARNING = '▲'
DOTTED_LOW = ':'

CATEGORIES = ['Uptime', 'Performance', 'HighUtilization', 'LowUtilization', 'Alerts', 'Errors']

# Not all of these sheets are implemented yet. Just the high-priority ones.
SHEETS = {
    "Site:Alerts:List": "Alerts",
    "Web:Requests:Response Time(sec)": "UrlRt",
    "ArcGIS:Site:Configuraton": "AGSSite",
    "ArcGIS:Services:Summary": "AGSServices",
    "ArcGIS:Requests:Count": "AGSServiceTr",
    "ArcGIS:Requests:Requests/sec": "AGSTh",
    #"ArcGIS:Services:Requests/sec": "AGSTrPeakHr",
    "ArcGIS:Services:Instances": "AGSFreeInstances",
    "ArcGIS:Services:Response Time(sec)": "AGSPerf",
    #"ArcGIS:Services:Response Time(sec)": "AGSPerfPeakHr",
    "Database:DB query:DB query": "DB",
    "Infrastructure:Summary:Summary": "SystemSum",
    "Infrastructure:CPU:Utilization(%)": "CPU",
    #"Infrastructure:CPU:Utilization(%)": "CPUPeakHr",
    "Infrastructure:Memory Physical:Utilization(%)": "PhysicalMem",
    #"Infrastructure:Memory Physical:Utilization(%)": "PhysicalMemHr",
    "Infrastructure:Memory Virtual:Utilization(%)": "VirtualMem",
    #"Infrastructure:Memory Virtual:Utilization(%)": "VirtualMemHr",
    "Infrastructure:Disk Utilization:Utilization(%)": "DiskIO",
    "Infrastructure:Disk Space:Utilization(%)": "DiskSpace",
    "Infrastructure:Network Received:mbps": "NetworkReceived",
    #"Infrastructure:Network Received:mbps": "NetworkReceivedPeakHr",
    "Infrastructure:Network Sent:mbps": "NetworkSent",
    #"Infrastructure:Network Sent:mbps": "NetworkSentPeakHr",
    "Infrastructure:Process CPU:Utilization(%)": "ProcessCPU",
    "Infrastructure:Process Memory Physical:Utilization(%)": "ProcessPhysicalMem",
    "Infrastructure:Process Memory Virtual:Utilization(%)": "ProcessVirtualMem",
    "Infrastructure:Process Count:Count": "ProcessCount",
    "Infrastructure:Process Count:Active": "ProcessActive",
    "Extensions:Extensions:Extensions": "Ext",
    "Site:Alerts:Configuraton": "AlertDefinition",
    "Site:Alerts:Not Configured": "NoAlertDefinition",
    "Site:Collection:Configuraton": "CollectionInfo",
    "Site:Counters:Configuraton": "CountersCategory",
    "Site:Collection Time:Utilization(%)": "CollectionTimes"
}

## Read Summary tab for warnings

In [ ]:
# Reading the summary tab
# all_warnings contains one entry for each Excel, with a list of the warnings in each
summary = []
all_warnings = []
for i in range(0, len(input_logs)):
    summary.append(input_logs[i]["Summary"])
    summary[i]["StatKey"] = summary[i].Category.astype(str) + \
        ':' + summary[i].Type.astype(str) + \
        ':' + summary[i].Name.astype(str)
    summary[i]['WarnCategory'] = ''
    warn = []
    crit = []
    for cat in CATEGORIES:
        summary[i].loc[summary[i][cat]==WARNING, 'WarnCategory'] = cat
        warn.append(summary[i][summary[i][cat]==WARNING])
        summary[i].loc[summary[i][cat]==CRITICAL, 'WarnCategory'] = cat
        crit.append(summary[i][summary[i][cat]==CRITICAL])

    warnings = pd.concat(warn)
    warnings["Severity"] = "Warning"
    criticals = pd.concat(crit)
    criticals["Severity"] = "Critical"

    all_warnings.append(pd.concat([criticals,warnings]))
#all_warnings

# Start Visualization
From here, each cell takes the warnings from one sheet and tries to visualize the data
with matplotlib or pandas plotting. The graphs are a first cut, and could stand a lot of improvement.

## Transaction Rates

In [ ]:
KEY = 'ArcGIS:Requests:Requests/sec'
SIMPLE_PLOT = True

if SIMPLE_PLOT:
    P_STATS = ['p5(Tr/sec)','p50(Tr/sec)','p75(Tr/sec)','p95(Tr/sec)','p99(Tr/sec)','Max(Tr/sec)']
    P_LABELS = ['p5','p50','p75','p95','p99', 'max']

    for i in range(0, len(all_warnings)):
        rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
        if rows.empty == False:
            title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
            stat_sheet = input_logs[i][SHEETS[KEY]]
            LIMIT_LOW = 1.0
            LIMIT_HIGH = 4.0

            fig, ax = plt.subplots(figsize=(10,5))
            ax.set_ylabel('Requests per second')

            for i in range(0, len(stat_sheet)):
                val = stat_sheet.loc[stat_sheet.index[i], 'p99(Tr/sec)']
                if val > LIMIT_LOW:
                    dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                    plt.plot(P_LABELS,
                             stat_sheet.loc[stat_sheet.index[i], P_STATS],
                             label=stat_sheet.loc[stat_sheet.index[i], 'Service'],
                             linestyle=dotted)
            plt.legend()
            plt.title(title)
            plt.show()

else: # Linear
    # EXPERIMENTAL: making the graph percentiles linear
    def get_p_values(df, i, suffix):
        vals = [None] * 100
        vals[0] = df.loc[df.index[i], f'Min({suffix})']
        vals[99] = df.loc[df.index[i], f'Max({suffix})']
        for p in [5,50,75,95,99]:
            vals[p-1] = df.loc[df.index[i], f'p{p}({suffix})']
        return np.array(vals).astype(np.double)

    # https://www.geeksforgeeks.org/how-to-make-a-bell-curve-in-python/
    
    x_range = np.linspace(start=0, stop=99, num=100)

    for i in range(0, len(all_warnings)):
        rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
        if rows.empty == False:
            title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
            stat_sheet = input_logs[i][SHEETS[KEY]]
            LIMIT_LOW = 1.0
            LIMIT_HIGH = 4.0

            fig, ax = plt.subplots(figsize=(10,5))
            ax.set_xlabel('p-value')

            for i in range(0, len(stat_sheet)):
                p_values = get_p_values(stat_sheet, i, 'Tr/sec')

                val = stat_sheet.loc[stat_sheet.index[i], 'p99(Tr/sec)']
                if val > LIMIT_LOW:
                    #display(p_values)
                    mask = np.isfinite(p_values)
                    dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                    plt.plot(x_range[mask],
                             p_values[mask],
                             label=stat_sheet.loc[stat_sheet.index[i], 'Service'],
                             linestyle=dotted, marker='o', markersize=3)
            plt.legend()
            plt.title(title)
            plt.show()



## Alerts

In [ ]:
KEY = "Site:Alerts:List"

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        grouped = stat_sheet.groupby(['Name', 'Note'], sort=False).size().reset_index(name='Count')
        grouped["Alert Source"] = grouped.Name.astype(str) + ' ' + grouped.Note.astype(str)
        grouped = grouped.sort_values('Count')
        
        fig, ax = plt.subplots(figsize=(10,len(grouped)))
        ax.set_xlabel('Count')
        # Restrict x axis ticks to whole numbers
        ax.xaxis.set_major_locator(pylab.MaxNLocator(integer = True))
        ax.barh(grouped['Alert Source'], grouped['Count'])
        plt.title(title)


## Service Instances

In [ ]:
KEY = "ArcGIS:Services:Instances"

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        index=[]
        data = dict(min=[], max=[], avgfree=[], minfree=[], maxfree=[], p95free=[])
        filtered = stat_sheet[(stat_sheet['Min(free)']==0) | (stat_sheet['Max(free)']==stat_sheet['MaxInstances'])]
        for i in range(0, len(filtered)):
            idx = filtered.index[i]
            index.append(filtered.loc[idx, 'Service'])
            data['min'].append(filtered.loc[idx, 'MinInstances'])
            data['max'].append(filtered.loc[idx, 'MaxInstances'])
            data['avgfree'].append(filtered.loc[idx, 'Avg(free)'])
            data['minfree'].append(filtered.loc[idx, 'Min(free)'])
            data['maxfree'].append(filtered.loc[idx, 'Max(free)'])
            data['p95free'].append(filtered.loc[idx, 'p95(free)'])

        fig, ax = plt.subplots(figsize=(10,10))
        ax.set_xlabel('Number of Instances')

        df = pd.DataFrame(data, index).sort_values(['max', 'min'])
        #print(df.loc[::-1])
        plt.barh(range(len(df)), df["max"]-df["min"], height=0.5, left=df["min"], color='pink')
        plt.plot(df["minfree"], range(len(df)), linestyle="", markersize=15, 
            marker=">", color="#FFFFFF", label="Min Free", markeredgecolor="k")
        plt.plot(df["maxfree"], range(len(df)), linestyle="", markersize=15, 
            marker="<", color="#FF0000", label="Max Free", markeredgecolor="k")
        plt.plot(df["p95free"], range(len(df)), linestyle="", markersize=10, 
            marker="o", color="#00FFFF", label="p95 Free", markeredgecolor="k")
        plt.plot(df["avgfree"], range(len(df)), linestyle="", markersize=10, 
            marker="o", color="#ff6600", label="Avg Free", markeredgecolor="k")

        plt.yticks(range(len(df)),df.index)
        plt.legend()
        plt.title(title + ' (Where min free is 0 or max free = max instances)')
        plt.grid(axis='x')
        plt.show()


## CPU Utilization

In [ ]:
KEY = 'Infrastructure:CPU:Utilization(%)'
P_STATS = ['p5(%)','p50(%)','p75(%)','p95(%)','p99(%)','Max(%)']
P_LABELS = ['p5','p50','p75','p95','p99', 'max']

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 10
        LIMIT_HIGH = 80

        fig, ax = plt.subplots(figsize=(10,5))
        ax.set_ylabel('Utilization (%)')

        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(%)']
            if val > LIMIT_LOW:
                dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                plt.plot(P_LABELS,
                         stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Machine'],
                         linestyle=dotted)
        plt.legend()
        plt.title(title)
        plt.show()



## Physical Memory

In [ ]:
KEY = 'Infrastructure:Memory Physical:Utilization(%)'
P_STATS = ['p5(%)','p50(%)','p75(%)','p95(%)','p99(%)','Max(%)']
P_LABELS = ['p5','p50','p75','p95','p99', 'max']

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 10
        LIMIT_HIGH = 80

        fig, ax = plt.subplots(figsize=(10,5))
        ax.set_ylabel('Utilization (%)')

        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(%)']
            if val > LIMIT_LOW:
                dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                plt.plot(P_LABELS,
                         stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Machine'],
                         linestyle=dotted)
        plt.legend()
        plt.title(title)
        plt.show()



## Virtual Memory

In [ ]:
KEY = 'Infrastructure:Memory Virtual:Utilization(%)'
P_STATS = ['p5(%)','p50(%)','p75(%)','p95(%)','p99(%)','Max(%)']
P_LABELS = ['p5','p50','p75','p95','p99', 'max']

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 10
        LIMIT_HIGH = 80
        
        fig, ax = plt.subplots(figsize=(10,5))
        ax.set_ylabel('Utilization (%)')

        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(%)']
            if val > LIMIT_LOW:
                dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                plt.plot(P_LABELS,
                         stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Machine'],
                         linestyle=dotted)
        plt.legend()
        plt.title(title)
        plt.show()

## Disk Space

In [ ]:
KEY = 'Infrastructure:Disk Space:Utilization(%)'
P_STATS = ['p5(%)','p50(%)','p75(%)','p95(%)','p99(%)','Max(%)']
P_LABELS = ['p5','p50','p75','p95','p99', 'max']

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 10
        LIMIT_HIGH = 80

        fig, ax = plt.subplots(figsize=(10,5))
        ax.set_ylabel('Utilization (%)')

        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(%)']
            if val > LIMIT_LOW:
                dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                plt.plot(P_LABELS,
                         stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Machine'],
                         linestyle=dotted)
        plt.legend()
        plt.title(title)
        plt.show()

## Network Received

In [ ]:
KEY = 'Infrastructure:Network Received:mbps'
P_STATS = ['p5(mbps)','p50(mbps)','p75(mbps)','p95(mbps)','p99(mbps)','Max(mbps)']
P_LABELS = ['p5','median','p75','p95','p99', 'max']
plt.rcParams["figure.figsize"] = (10,5)

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 10
        LIMIT_HIGH = 80
        ind = np.arange(len(P_LABELS))
        width = 0.2
        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(mbps)']
            if val > LIMIT_LOW:
                plt.bar(x=(ind-width*i), tick_label=P_LABELS, width=width,
                         height=stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Machine'])

        plt.legend()
        plt.title(title)
        plt.show()

## Network Sent

In [ ]:
KEY = 'Infrastructure:Network Sent:mbps'
P_STATS = ['p5(mbps)','p50(mbps)','p75(mbps)','p95(mbps)','p99(mbps)','Max(mbps)']
P_LABELS = ['p5','p50','p75','p95','p99', 'max']
plt.rcParams["figure.figsize"] = (10,5)

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 10
        LIMIT_HIGH = 80
        ind = np.arange(len(P_LABELS))
        width = 0.2
        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(mbps)']
            if val > LIMIT_LOW:
                plt.bar(x=(ind-width*i), tick_label=P_LABELS, width=width,
                         height=stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Machine'])
        plt.legend()
        plt.title(title)
        plt.show()

## Response Times

In [ ]:
KEY = 'ArcGIS:Services:Response Time(sec)'
P_STATS = ['p5(sec)','p50(sec)','p75(sec)','p95(sec)','p99(sec)','Max(sec)']
P_LABELS = ['p5','p50','p75','p95','p99', 'max']
plt.rcParams["figure.figsize"] = (10,5)

for i in range(0, len(all_warnings)):
    rows = all_warnings[i].loc[all_warnings[i]['StatKey']==KEY]
    if rows.empty == False:
        title = f'{KEY} {input_logs[i]["Inputs"].at[6, "Value"].strftime("%Y-%m-%d")}'
        stat_sheet = input_logs[i][SHEETS[KEY]]
        LIMIT_LOW = 1
        LIMIT_HIGH = 2

        for i in range(0, len(stat_sheet)):
            val = stat_sheet.loc[stat_sheet.index[i], 'p95(sec)']
            if val > LIMIT_LOW:
                dotted = None if (val > LIMIT_HIGH) else DOTTED_LOW
                plt.plot(P_LABELS,
                         stat_sheet.loc[stat_sheet.index[i], P_STATS],
                         label=stat_sheet.loc[stat_sheet.index[i], 'Service'],
                         linestyle=dotted)
        plt.legend()
        plt.title(title)
        plt.show()